In [ ]:
#Imports
from keras import layers, models, optimizers
from keras.layers import Dropout
from keras_preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
import glob
import os
import shutil
import random as rnd
import matplotlib.pyplot as plt
import PIL
import numpy as np

In [ ]:
#Flower type
def get_flowertype(string_to_overwrite):
    flowers = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
    for flower in flowers:
        if flower in string_to_overwrite:
            return flower

In [ ]:
# Paths
org_dir = os.getcwd()+'/data/flowers'
path = org_dir+'/*'
new_dir = os.getcwd()+'/data/flowers_split'

training_dir = os.path.join(new_dir, 'train')
validation_dir = os.path.join(new_dir, 'validation')
test_dir = os.path.join(new_dir, 'test')

In [ ]:
# Split, OBS only run once
images = []
for folder in glob.glob(path):
    flower_type = get_flowertype(path+folder)
    for file in glob.glob(folder+'/*'):
        if flower_type is None or os.path.basename(file) is None:
            continue
        images.append((os.path.basename(file), flower_type))
rnd.shuffle(images)
size = len(images)

training_set = images[:int(size*0.6)]
validation_set = images[int(size*0.6):int(size*0.8)]
test_set = images[int(size*0.8):]

for im, flower_type in training_set:
    src = os.path.join(org_dir, flower_type, im)
    dst = os.path.join(training_dir, flower_type, im)
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copyfile(src, dst)

for im, flower_type in validation_set:
    src = os.path.join(org_dir, flower_type, im)
    dst = os.path.join(validation_dir, flower_type, im)
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copyfile(src, dst)

for im, flower_type in test_set:
    src = os.path.join(org_dir, flower_type, im)
    dst = os.path.join(test_dir, flower_type, im)
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copyfile(src, dst)
    

In [ ]:
# If need size again but don't want to split.
size = 0
for folder in glob.glob(path):
    for file in glob.glob(folder+'/*'):
        size += 1
print(size)

In [ ]:
# Model 1
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.rmsprop(lr=1e-4),
              metrics=['acc'])
model.summary()

In [ ]:
# Load weights
model.load_weights('weights1.h5')

In [ ]:
# Parameters
b_size = 64
epochs = 5
steps = int(np.ceil(size / b_size))
val_steps = int(np.ceil(size*0.2 / b_size))

In [ ]:
# To use transformed data use this
datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=40,
            width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True,
            fill_mode='nearest')

In [ ]:
# If normal data use this
datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Generate generators
training_generator = datagen.flow_from_directory(new_dir + '/train', target_size=(150, 150),
                                                         batch_size=b_size)

validation_generator = test_datagen.flow_from_directory(new_dir + '/validation', target_size=(150, 150),
                                                       batch_size=b_size)

test_generator = test_datagen.flow_from_directory(new_dir + '/test', target_size=(150, 150), batch_size=b_size)

In [ ]:
# Fit generator
history = model.fit_generator(training_generator,
                                    steps_per_epoch=steps,
                                    epochs=epochs,
                                    validation_data=validation_generator,
                                    validation_steps=val_steps)

In [ ]:
# Evaluate
test_loss, test_acc = model.evaluate_generator(test_generator, steps=len(test_generator))
print('Test accurucy: ', test_acc)

In [ ]:
#Save weights
model.save('weights1.h5')

In [ ]:
# Plot
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# Using pretrained network
#Part 1

In [ ]:
# Model 2
conv_base = VGG16(weights='imagenet',
                          include_top=False,
                          input_shape=(150, 150, 3))
model_2 = models.Sequential()
model_2.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model_2.add(layers.Dropout(0.5))
model_2.add(layers.Dense(5, activation='softmax'))
model_2.compile(loss='categorical_crossentropy', optimizer=optimizers.rmsprop(lr=1e-4), metrics=['acc'])

In [ ]:
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count, 5))

    generator = datagen.flow_from_directory(directory, target_size=(150, 150), batch_size=b_size)
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * b_size: (i + 1) * b_size] = features_batch
        labels[i * b_size: (i + 1) * b_size] = labels_batch
        i += 1
        if i * b_size >= sample_count:
            break
        return features, labels

In [ ]:
#Extract and reshape
train_features, train_labels = extract_features(training_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

In [ ]:
# Fit
history = model_2.fit(train_features, train_labels, epochs=epochs,
                    batch_size=b_size,
                    validation_data=(validation_features, validation_labels))

In [ ]:
# Evaluate
test_loss, test_acc = model_2.evaluate(test_features, test_labels)
print(test_acc)

In [ ]:
# Pretrained networks
# Part 2
# Model 3
model_3 = models.Sequential()
model_3.add(conv_base)
model_3.add(layers.Flatten())
model_3.add(layers.Dense(256, activation='relu'))
model_3.add(layers.Dense(5, activation='softmax'))

conv_base.trainable = False
model_3.compile(loss='categorical_crossentropy', optimizer=optimizers.rmsprop(lr=1e-4), metrics=['acc'])

In [ ]:
# Fit
history = model_3.fit_generator(training_generator,
                                      steps_per_epoch=steps,
                                      epochs=epochs,
                                      validation_data=validation_generator,
                                      validation_steps=val_steps)

In [ ]:
# Eval
test_loss, test_acc = model_3.evaluate_generator(test_generator)
print('Test accurucy: ', test_acc)

In [ ]:
# Plot
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()